In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns  
import matplotlib.pyplot as plt
import cv2
from glob import glob
from PIL import Image
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0

In [10]:
SIZE = 32
NUM_CLASSES = 7

# Load Data
skin_df = pd.read_csv('/Python/Ham10000/archive/HAM10000_metadata.csv')
le = LabelEncoder()
skin_df['label'] = le.fit_transform(skin_df['dx'])

# Image Paths
image_path = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join('D://Python/Ham10000/archive/', '*', '*.jpg'))}
skin_df['path'] = skin_df['image_id'].map(image_path.get)
skin_df['image'] = skin_df['path'].map(lambda x: np.asarray(Image.open(x).resize((SIZE, SIZE))))

# Data Preparation
X = np.asarray(skin_df['image'].tolist()) / 255.0
Y = to_categorical(skin_df['label'], num_classes=NUM_CLASSES)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

# CNN Model

In [11]:

# CNN Model
def create_cnn_model():
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(SIZE, SIZE, 3)),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.3),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.3),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [12]:
def create_pretrained_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
    base_model.trainable = False
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [13]:

# Train CNN
cnn_model = create_cnn_model()
cnn_model.fit(x_train, y_train, epochs=20, batch_size=16, validation_data=(x_test, y_test))


Epoch 1/20


c:\Users\ganes\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


470/470 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.6654 - loss: 1.0470 - val_accuracy: 0.6833 - val_loss: 0.9444
Epoch 2/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.6870 - loss: 0.8946 - val_accuracy: 0.6877 - val_loss: 0.8527
Epoch 3/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.7017 - loss: 0.8372 - val_accuracy: 0.7029 - val_loss: 0.7952
Epoch 4/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.7006 - loss: 0.8275 - val_accuracy: 0.7025 - val_loss: 0.7869
Epoch 5/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.7231 - loss: 0.7650 - val_accuracy: 0.6981 - val_loss: 0.8079
Epoch 6/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.7221 - loss: 0.7502 - val_accuracy: 0.7141 - val_loss: 0.7646
Epoch 7/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.7301 - loss: 0.7172 - val_accuracy: 0.7220 - val_loss: 0.7406
Epoch 8/20
470/470 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.7259 - loss: 0.7240 - val_accuracy: 0.722

In [14]:
score = cnn_model.evaluate(x_test, y_test)
print('Test accuracy:', score[1])


79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7076 - loss: 0.8105
Test accuracy: 0.7336261868476868


In [ ]:

# Train Pre-trained Model
pretrained_model = create_pretrained_model()
pretrained_model.fit(x_train, y_train, epochs=10, batch_size=16, validation_data=(x_test, y_test))


Epoch 1/10
470/470 ━━━━━━━━━━━━━━━━━━━━ 51s 101ms/step - accuracy: 0.6473 - loss: 1.3406 - val_accuracy: 0.6661 - val_loss: 1.1024
Epoch 2/10
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6634 - loss: 1.1652